May: How Mackenzie Canyon fits into the established scaling for depth of upwelling and upwelling flux.

In [1]:
import netCDF4 as nc
import xarray as xr
import numpy as np
from scipy import interpolate
from salishsea_tools import gsw_calls
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = '/ocean/imachuca/Canyons/results_mackenzie/idealized/upwelling_events/eventB_pulse10_bdy' 
gridT = xr.open_dataset(path+'/GYRE_1h_20170101_20170120_grid_T.nc')
gridU = xr.open_dataset(path+'/GYRE_1h_20170101_20170120_grid_U.nc')
gridV = xr.open_dataset(path+'/GYRE_1h_20170101_20170120_grid_V.nc')
gridW = xr.open_dataset(path+'/GYRE_1h_20170101_20170120_grid_W.nc')

In [3]:
nav_lat = np.asfarray(gridT['nav_lat'])
deptht = np.asfarray(gridT['deptht'])

In [4]:
initial_sal = nc.Dataset('../../mackenzie_canyon/conditions/NEMO_files/salinity/salinity_doubled_20.nc')['vosaline']
initial_tem = nc.Dataset('../../mackenzie_canyon/conditions/NEMO_files/temperature/temperature_doubled_20.nc')['votemper']

# Calculate N

The mean latitude and all depth levels are used to find the pressure profile.

In [5]:
dep = -1 * deptht
lat = np.ones(1) * np.mean(nav_lat)

print(dep.shape, lat.shape, lat)

p = gsw_calls.generic_gsw_caller('gsw_p_from_z.m', [dep, lat], matlab_gsw_dir='/ocean/rich/home/matlab/gsw3')

(20,) (1,) [ 70.71705431]


In [6]:
SA = initial_sal[0,:,0,0]
CT = initial_tem[0,:,0,0]

print(SA.shape, CT.shape, p.shape)

rho = gsw_calls.generic_gsw_caller('gsw_rho_CT_exact.m', [SA, CT, p], matlab_gsw_dir='/ocean/rich/home/matlab/gsw3')

(20,) (20,) (20,)


In [7]:
d_rho0 = np.diff(rho)
d_dep0 = np.diff(dep)

# diff does n-1 minus n so we lose the last element, thus extrapolate
f_rho = interpolate.interp1d(np.arange(len(d_rho0)), d_rho0, fill_value='extrapolate')
f_dep = interpolate.interp1d(np.arange(len(d_dep0)), d_dep0, fill_value='extrapolate')

d_rho = f_rho(np.arange(len(d_rho0)+1))
d_dep = f_dep(np.arange(len(d_dep0)+1))

N = np.sqrt((-9.81 / rho)*(d_rho / d_dep))

print(np.mean(N))

0.0161591616779
